In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import time

#xgboost
from xgboost import XGBClassifier

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score, RandomizedSearchCV, StratifiedKFold


from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_auc_score,roc_curve
from sklearn.base import BaseEstimator, TransformerMixin, clone

from scipy.stats import ttest_rel
from scipy.stats import randint, uniform

#hiperparamentros search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform,loguniform


# Importações locais
from setup_notebook import setup_path
setup_path()
from src.model_utils import *
from src.preprocess_utils_diab import *
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
start_inicial = time.time()


#Processo iniciado em: 09:42:28


## 1. Load Data & Pipeline

In [2]:
BASE = Path.cwd().parent

PP2 = joblib.load(BASE/'src'/'preprocess_diabetes_v1.2.joblib')['preprocessador']

DATA_DIR = BASE/"data"/"raw"
X_train = pd.read_csv(DATA_DIR/"X_train_raw.csv")
X_test  = pd.read_csv(DATA_DIR/"X_test_raw.csv")
y_train = pd.read_csv(DATA_DIR/"y_train_raw.csv").values.ravel()
y_test  = pd.read_csv(DATA_DIR/"y_test_raw.csv").values.ravel()
mtd_scoring='roc_auc'

## 2.Baseline

In [3]:
def  best_threshold(y_test,yprob):
    # Threshold search
    thresholds = np.linspace(0.3, 0.7, 41)
    best_val = 0.5
    max_acc = 0
    for t in thresholds:
        acc = accuracy_score(y_test, yprob > t)
        if acc > max_acc:
            max_acc = acc
            best_val = t
    return best_val,max_acc

def print_hiper(search_1_best_params):
    
    print("🎯 Melhores Hiperparâmetros")
    print("-" * 50)
    for param, value in search_1_best_params.items():
        param_name = param.replace('model__', '').replace('_', ' ').title()
        print(f"• {param_name:<25} : {value}")
    print("-" * 50)

#=====================================================================
# model Baseline
print("\n#Processo iniciado em:", time.strftime("%H:%M:%S"))
model_base = XGBClassifier(objective='binary:logistic',eval_metric='logloss',
                            random_state=42)

pipe_base  = pipe_models(model_base, PP2)

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

pipe_base.fit(X_train, y_train)
y_probs0 = pipe_base.predict_proba(X_test)[:, 1]

threshold_0,max_acc_0=best_threshold(y_test,y_probs0)
auc_score_0 = roc_auc_score(y_test, y_probs0)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_0:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_0:.4f}")
print(f"⭐ AUC Score                : {auc_score_0:.4f}")
print(f"{'='*70}")
print_hiper(model_base.get_params())

print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))


#Processo iniciado em: 09:42:29
🎯 Melhor Threshold         : 0.510
📈 Melhor Acurácia de Teste : 0.6805
⭐ AUC Score                : 0.7209
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Objective                 : binary:logistic
• Base Score                : None
• Booster                   : None
• Callbacks                 : None
• Colsample Bylevel         : None
• Colsample Bynode          : None
• Colsample Bytree          : None
• Device                    : None
• Early Stopping Rounds     : None
• Enable Categorical        : False
• Eval Metric               : logloss
• Feature Types             : None
• Gamma                     : None
• Grow Policy               : None
• Importance Type           : None
• Interaction Constraints   : None
• Learning Rate             : None
• Max Bin                   : None
• Max Cat Threshold         : None
• Max Cat To Onehot         : None
• Max Delta Step            : None
• Max Depth                 : No

## 3.Buscas por hiperparamentros


In [4]:

print("#Processo Iniciado em:", time.strftime("%H:%M:%S"))
# Exploratório
param_dist_1 = {
    # 1. Controle de Aprendizado
    'model__n_estimators': randint(100, 3000), 
    'model__learning_rate':  loguniform(0.003, 0.3),
    
    # 2. Complexidade e Profundidade
    'model__max_depth': randint(3, 13),
    'model__min_child_weight': randint(1, 11),
    
    # 3. Robustez contra Overfitting (Stochastic Gradient Boosting)
    'model__subsample': uniform(0.5, 0.5),
    'model__colsample_bytree': uniform(0.5, 0.5),
    
    # 4. Regularização (Penalidades)
    'model__gamma': uniform(0, 0.5),
    'model__reg_alpha': uniform(0, 5), 
    'model__reg_lambda': uniform(1, 8)   
}

cv_s = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
numero_itera=50


search_1 = RandomizedSearchCV(
    pipe_base, param_dist_1,
    n_iter=numero_itera, cv=cv_s,
    scoring=mtd_scoring,
    random_state=42, verbose=1
)

start = time.time()
search_1.fit(X_train, y_train)
end = time.time()

best_1 = search_1.best_estimator_

y_probs1 = best_1.predict_proba(X_test)[:, 1]

threshold_1,max_acc_1=best_threshold(y_test,y_probs1)
auc_score_1 = roc_auc_score(y_test, y_probs1)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_1:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_1:.4f}")
print(f"⭐ AUC Score                : {auc_score_1:.4f}")
print(f"{'='*70}")
print_hiper(search_1.best_params_)

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

#Processo Iniciado em: 09:42:34
Fitting 3 folds for each of 50 candidates, totalling 150 fits
🎯 Melhor Threshold         : 0.500
📈 Melhor Acurácia de Teste : 0.6846
⭐ AUC Score                : 0.7264
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Colsample Bytree          : 0.9077307142274171
• Gamma                     : 0.35342867192380856
• Learning Rate             : 0.08612626044586402
• Max Depth                 : 3
• Min Child Weight          : 5
• N Estimators              : 2378
• Reg Alpha                 : 3.2553851275097223
• Reg Lambda                : 8.319677404350246
• Subsample                 : 0.9250192888948996
--------------------------------------------------
Tempo total: 4883.02 segundos
Tempo por iteração: 97.66 segundos

#Processo finalizado em: 11:03:59


In [5]:
print("#Processo Iniciado em:", time.strftime("%H:%M:%S"))
def generate_refined_grid(best_params):
    """
    Gera um grid refinado automaticamente baseado nos melhores parâmetros
    do Random Search exploratório (XGBoost).
    """
    # ---------------------------
    # 1. n_estimators
    # ---------------------------
    n_best = best_params['model__n_estimators']
    n_min  = int(n_best * 0.8)
    n_max  = int(n_best * 1.2)

    # ---------------------------
    # 2. learning_rate
    # ---------------------------
    lr_best = best_params['model__learning_rate']
    lr_st   = lr_best * 0.75
    lr_dist = 0.3  # Amplitude definida no seu código original

    # ---------------------------
    # 3. max_depth (Tratamento de None)
    # ---------------------------
    depth_best = best_params['model__max_depth']
    if depth_best is not None:
        depth_list = [max(1, depth_best - 1), depth_best, depth_best + 1]
    else:
        depth_list = [None, 10, 20]

    # ---------------------------
    # 4. min_child_weight
    # ---------------------------
    mcw_best = best_params['model__min_child_weight']
    mcw_list = [max(0, mcw_best - 1), mcw_best, mcw_best + 1]

    # ---------------------------
    # 5. subsample
    # ---------------------------
    ss_best = best_params['model__subsample']
    ss_st   = ss_best * 0.75
    ss_dist = 0.3

    # ---------------------------
    # 6. colsample_bytree
    # ---------------------------
    cb_best = best_params['model__colsample_bytree']
    cb_st   = cb_best * 0.75
    cb_dist = 0.3

    # ---------------------------
    # 7. gamma
    # ---------------------------
    gamma_best = best_params['model__gamma']
    gamma_st   = gamma_best * 0.75
    gamma_dist = 0.5

    # ---------------------------
    # 8. regularização (alpha e lambda)
    # ---------------------------
    alpha_best  = best_params['model__reg_alpha']
    lambda_best = best_params['model__reg_lambda']

    alpha_st   = alpha_best * 0.75
    alpha_dist = 0.5

    lambda_st   = lambda_best * 0.75
    lambda_dist = 0.5

    # ---------------------------
    # PRINTS DIAGNÓSTICOS
    # ---------------------------
    print(f"{'='*30} NOVO GRID DE REFINAMENTO (XGBoost) {'='*30}")
    print(f"🔹 n_estimators        : {n_min} - {n_max}")
    print(f"🔹 learning_rate       : {lr_st:.5f} - {lr_st + lr_dist:.5f}")
    print(f"🔹 max_depth           : {depth_list}")
    print(f"🔹 min_child_weight    : {mcw_list}")
    print(f"🔹 subsample           : {ss_st:.3f} - {min(1.0, ss_st + ss_dist):.3f}")
    print(f"🔹 colsample_bytree    : {cb_st:.3f} - {min(1.0, cb_st + cb_dist):.3f}")
    print(f"🔹 gamma               : {gamma_st:.4f} - {gamma_st + gamma_dist:.4f}")
    print(f"🔹 reg_alpha           : {alpha_st:.4f} - {alpha_st + alpha_dist:.4f}")
    print(f"🔹 reg_lambda          : {lambda_st:.4f} - {lambda_st + lambda_dist:.4f}")
    print(f"{'='*86}\n")

    # ---------------------------
    # GRID FINAL
    # ---------------------------
    refined_grid = {
        'model__n_estimators': randint(n_min, n_max + 1),
        'model__learning_rate': uniform(lr_st, lr_dist),
        'model__max_depth': depth_list,
        'model__min_child_weight': mcw_list,
        'model__subsample': uniform(ss_st, ss_dist),
        'model__colsample_bytree': uniform(cb_st, cb_dist),
        'model__gamma': uniform(gamma_st, gamma_dist),
        'model__reg_alpha': uniform(alpha_st, alpha_dist),
        'model__reg_lambda': uniform(lambda_st, lambda_dist)
    }

    return refined_grid

# # Aplicação automática
param_dist_2 = generate_refined_grid(search_1.best_params_)

numero_itera = 30
search_2 = RandomizedSearchCV(
    pipe_base, param_dist_2,
    n_iter=numero_itera, cv=cv_s, 
    scoring=mtd_scoring,
    random_state=42, verbose=1
)

start = time.time()
search_2.fit(X_train, y_train)
end = time.time()

best_2 = search_2.best_estimator_
y_probs2 = best_2.predict_proba(X_test)[:, 1]

threshold_2,max_acc_2=best_threshold(y_test,y_probs2)
auc_score_2 = roc_auc_score(y_test, y_probs2)

print(f"{'='*70}")
print(f"🎯 Melhor Threshold         : {threshold_2:.3f}")
print(f"📈 Melhor Acurácia de Teste : {max_acc_2:.4f}")
print(f"⭐ AUC Score                : {auc_score_2:.4f}")
print(f"{'='*70}")
print_hiper(search_2.best_params_)

print(f"Tempo total: {end-start:.2f} segundos")
print(f"Tempo por iteração: {(end-start)/numero_itera:.2f} segundos")
print("\n#Processo finalizado em:", time.strftime("%H:%M:%S"))

#Processo Iniciado em: 11:03:59
============================== NOVO GRID DE REFINAMENTO (XGBoost) ==============================
🔹 n_estimators        : 1902 - 2853
🔹 learning_rate       : 0.06459 - 0.36459
🔹 max_depth           : [2, 3, 4]
🔹 min_child_weight    : [4, 5, 6]
🔹 subsample           : 0.694 - 0.994
🔹 colsample_bytree    : 0.681 - 0.981
🔹 gamma               : 0.2651 - 0.7651
🔹 reg_alpha           : 2.4415 - 2.9415
🔹 reg_lambda          : 6.2398 - 6.7398

Fitting 3 folds for each of 30 candidates, totalling 90 fits
🎯 Melhor Threshold         : 0.510
📈 Melhor Acurácia de Teste : 0.6846
⭐ AUC Score                : 0.7263
🎯 Melhores Hiperparâmetros
--------------------------------------------------
• Colsample Bytree          : 0.8814945120848314
• Gamma                     : 0.5554148146610838
• Learning Rate             : 0.17627952530292096
• Max Depth                 : 2
• Min Child Weight          : 6
• N Estimators              : 2054
• Reg Alpha                 : 2.444

In [6]:
end_final = time.time()
print(f"Tempo final: {(end_final-start_inicial)/60:.2f} min")

Tempo final: 119.76 min


In [9]:
#Salvar Hiperparametros joblib

# joblib.dump(search_1.best_estimator_, 'modelo_XGB_final_randsearch.'+mtd_scoring+'_v1.2.joblib')
# joblib.dump(search_2.best_estimator_, 'modelo_XGB_final_refine.'+mtd_scoring+'_v1.2.joblib')

['modelo_XGB_final_refine.roc_auc_v1.2.joblib']